# Ejemplo CVA

Se calcula el CVA de un swap.

## Librerías

Las funciones relacionadas con el modelo de Hull-White que definimos y utilizamos en los notebooks 13 y 14 han sido mejoradas y encapsuladas en el módulo `hull_white.py`.

La función `bono_tasa_fija` se trasladó al nuevo módulo `instruments.py`.

In [72]:
from modules import instruments as inst
from modules import hull_white as hw
from modules import auxiliary as aux
import pandas as pd
import numpy as np

## Data Curva

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva['t'] = df_curva['plazo'] / 365.0

In [4]:
df_curva.head()

,plazo,tasa,df,t
0,1,0.000811,0.999998,0.002740
1,7,0.000841,0.999984,0.019178
2,14,0.000780,0.999970,0.038356
3,21,0.000774,0.999955,0.057534
4,33,0.000781,0.999929,0.090411


## Funciones para el Modelo de HW

- `zrate`: es el cubic spline construido a partir de las columnas `t` y `tasa` de `df_curva`.
- `hwz`: es una versión más sencilla de la función `zero_hw` que sólo requiere llos parámetros `r`, `t` y `T`.
- `theta`: es la misma función que ya utilizamos.

In [53]:
zrate, hwz, theta = hw.get_zrate_hwzero_and_theta(df_curva['t'], df_curva['tasa'], gamma, sigma)

## Simulación

Se definen los parámetros requeridos, tanto del modelo como de la simulación.

In [54]:
gamma = 1.0
sigma = .0025
r0 = zrate(.0001)
num_sims = 1000
num_steps = 528

Se obtienen las trayectorias.

In [55]:
time_steps, paths = hw.sim_hw_many(gamma, sigma, theta, r0, 1000, 528, seed=1000)

Se extrae el elemento correspondiente a 6M, 1Y, 18M de cada simulación:

In [58]:
last_r = [(s[132], s[264], s[396]) for s in paths]
last_r[0]

(0.00023065735784508546, -0.0022857834272125696, -0.0010278696736560555)

## Cálculo de la *EE* de un Swap

Para el swap vamos a utilizar una función sencilla para la pata fija y consideraremos que el valor presente de la pata flotante siempre es igual al nocional. Consideremos un swap a 2Y.

In [59]:
swap_fija = inst.bono_tasa_fija(0, .5, 4, .001)

In [60]:
swap_fija

[(0.5, 0.05), (1.0, 0.05), (1.5, 0.05), (2.0, 100.05)]

Se define una función que construye una función que valoriza el swap a partir de $t$ y $r_t$.

In [61]:
def valorizador_swap(pata_fija, df_function):
    def valor(r, t):
        result = 0.0
        for e in pata_fija:
            if e[0] > t:
                result += e[1] * df_function(r, t, e[0])
        return result - 100.0 if result > 0.0 else 0.0
    return valor

Probemos:

In [62]:
vswap = valorizador_swap(swap_fija, hwz)

In [63]:
vswap(r0, 1.5)

0.017069551127349314

Valoricemos el swap en 6M más en cada simulación:

In [69]:
valor_swap_6m = [vswap(r[0], .5) for r in last_r]
valor_swap_6m[0: 10]

[0.1087151035156495,
 -0.06891838127835115,
 -0.08828135334222509,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 -0.10416145596083481,
 -0.012453256721840944]

Mejor aún, podemos calcular directamente la exposición.

In [71]:
e_swap_6m = [max(vswap(r[0], .5), 0) for r in last_r]
e_swap_6m[0: 10]

[0.1087151035156495,
 0,
 0,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 0,
 0]

Por lo tanto, la exposición esperada en 6M es:

In [74]:
ee_swap_6m = np.average(e_swap_6m)
print(f'La EE del swap en 6M es: {ee_swap_6m: ,.2f}')

La EE del swap en 6M es:  0.09
